# MNIST

This example trains a GMLS convolutional neural network on the MNIST dataset. We apply convolution -> batch renormalization three times followed by dense layers. This architecture is similar to strided convolutional networks. Each convolution layer downsamples the point cloud while increasing the number of channels

In [1]:
import gmlsnets_tensorflow as gnets
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import itertools

In [2]:
#Loads mnist data. Normalizes the image data. Computes one-hot encoding of labels
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = (1.0*train_images/255).astype('float32')
test_images = (1.0*test_images/255).astype('float32')
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [3]:
# number of pixels per side
n=28

# channel widths
chans1=8
chans2=16
chans3=32

# constructs 2d Taylor basis of order 2
order = 2
dim = 2
fP = gnets.bases.Taylor(dim,order)

In [4]:
# Point clouds and epsilon balls to compute neighbors
x1 = 1.0*n*np.reshape(list(itertools.product(np.linspace(0,1,n),np.linspace(0,1,n))),
            (n**2,2)).astype('float32')
eps1 = (1.0*n/n)*(order+0.5)
x2 = 1.0*n*np.reshape(list(itertools.product(np.linspace(0,1,n//2),np.linspace(0,1,n//2))),
            ((n//2)**2,2)).astype('float32')
eps2 = (2.0*n/n)*(order+0.5)
x3 = 1.0*n*np.reshape(list(itertools.product(np.linspace(0,1,n//4),np.linspace(0,1,n//4))),
            ((n//4)**2,2)).astype('float32')
eps3 = (4.0*n/n)*(order+0.5)
x4 = 1.0*n*np.reshape(list(itertools.product(np.linspace(0,1,n//8),np.linspace(0,1,n//8))),
            ((n//8)**2,2)).astype('float32')
eps4 = (8.0*n/n)*(order+0.5)

In [5]:
#Constructs the model

model = tf.keras.Sequential()
model.add(tf.keras.layers.Reshape((n**2,1),input_shape=(n,n)))

model.add(gnets.MFConvLayer(x1,x2,fP,gnets.weightfuncs.sixth,eps1,chans1,activation='elu'))
model.add(tf.keras.layers.BatchNormalization(-1))

model.add(gnets.MFConvLayer(x2,x3,fP,gnets.weightfuncs.sixth,eps2,chans2,activation='elu'))
model.add(tf.keras.layers.BatchNormalization(-1))

model.add(gnets.MFConvLayer(x3,x4,fP,gnets.weightfuncs.sixth,eps4,chans3,activation='elu'))
model.add(tf.keras.layers.BatchNormalization(-1))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(100, activation='elu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [6]:
#trains model
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])

model.fit(train_images, train_labels,
          batch_size=501,
          epochs=20,
          verbose=1,)

Train on 60000 samples
Epoch 1/20


/home/rgpatel/.local/lib/python3.5/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/rgpatel/.local/lib/python3.5/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/rgpatel/.local/lib/python3.5/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/rgpatel/.local/lib/python3.5/site-packages/tensorflow_core/python/framework/indexed_slic

60000/60000 [==============================] - 11s 186us/sample - loss: 0.8602 - accuracy: 0.7236
Epoch 2/20
60000/60000 [==============================] - 9s 157us/sample - loss: 0.3574 - accuracy: 0.8894
Epoch 3/20
60000/60000 [==============================] - 9s 157us/sample - loss: 0.2512 - accuracy: 0.9222
Epoch 4/20
60000/60000 [==============================] - 9s 153us/sample - loss: 0.1940 - accuracy: 0.9389
Epoch 5/20
60000/60000 [==============================] - 9s 151us/sample - loss: 0.1549 - accuracy: 0.9517
Epoch 6/20
60000/60000 [==============================] - 9s 151us/sample - loss: 0.1276 - accuracy: 0.9595
Epoch 7/20
60000/60000 [==============================] - 9s 151us/sample - loss: 0.1066 - accuracy: 0.9668
Epoch 8/20
60000/60000 [==============================] - 9s 151us/sample - loss: 0.0912 - accuracy: 0.9719
Epoch 9/20
60000/60000 [==============================] - 9s 152us/sample - loss: 0.0791 - accuracy: 0.9758
Epoch 10/20
60000/60000 [=============

In [7]:
print('test set accuracy:',model.evaluate(test_images,test_labels, verbose=0)[1])

test set accuracy: 0.9737
